# Análise Comparativa de Dados

In [31]:
from IPython.display import display, Markdown
import joblib
import numpy as np
import pandas as pd

from sklearn.experimental import enable_iterative_imputer  # Necessário para habilitar o IterativeImputer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression

import seaborn as sns
import matplotlib.pyplot as plt

## 1. Obtenção dos Dados

In [4]:
df = pd.read_csv("../data/raw/data.csv")
df_dict = pd.read_csv("../data/external/dictionary.csv")
df_dict

,variavel,descricao,tipo,subtipo
0,total_bill,Valor total da conta,Quantitativa,Contínua
1,tip,Valor da gorjeta,Quantitativa,Contínua
2,sex,"Gênero do cliente pagante (Male = Masculino, F...",Qualitativa,Nominal
3,smoker,Informa se algum cliente da mesa é fumante (Ye...,Qualitativa,Nominal
4,day,Dia da semana em que ocorreu o serviço,Qualitativa,Nominal
5,time,Horário em que ocorreu o serviço (Lunch = Almo...,Qualitativa,Nominal
6,size,Quantidade de pessoas na mesa (1-6),Quantitativa,Discreta


## 2. Preparação de Dados

In [6]:
print(df['day'].unique())

['Sun' 'Sat' 'Thur' 'Fri']


In [43]:
target_column = 'tip' #que queremos prever

#lista de variáveis nominais
nominal_columns = (
    df_dict
    .query("subtipo == 'Nominal'")
    .variavel
    .to_list()
)

#lista de variáveis continuas
continuous_columns = (
    df_dict
    .query("subtipo == 'Contínua' and variavel != @target_column")
    .variavel
    .to_list()
)

discreet_columns = (
    df_dict
    .query("subtipo == 'Discreta'")
    .variavel
    .to_list()
)

x = df.drop(columns=[target_column], axis=1)
y = df[target_column]

In [45]:
#tratamento de dados discrepantes e possíveis dados faltantes 

nominal_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='most_frequent')), # tratamento de dados faltantes
    ('encoding', OneHotEncoder(sparse_output=False, drop='first')), # codificação de variáveis
    ('normalization', StandardScaler()) # normalização de dados
])

continuous_preprocessor = Pipeline([
    ('missing', IterativeImputer(estimator=LinearRegression())), #tratamento de dados faltantes
    ('normalization', StandardScaler()) # normalização de dados
])


discreet_preprocessor = Pipeline([
    ('missing', IterativeImputer(estimator=LinearRegression())), # tratamento de dados faltantes
    ('normalization', StandardScaler()) # normalização de dados
])

preprocessor = ColumnTransformer([
    ('nominal', nominal_preprocessor, nominal_columns),
    ('continuous', continuous_preprocessor, continuous_columns),
    ('discreet', discreet_preprocessor, discreet_columns)
])